# Create Clustering Matrices
## Dependencies

In [1]:
# !pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import joblib
import os
import json

### sklearn dependencies
import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.model_selection import GridSearchCV
from scipy import sparse

### suppress warnings for now
import warnings
warnings.filterwarnings("ignore", category = RuntimeWarning,
                        message = "overflow encountered in long_scalars")

In [3]:
filepath = '/home/datallah/datallah-jaymefis-gibsonce/'
sample_size = 'one'
random_state = 42
stop = {'a', 'about', 'above', 'after', 'again', 'against', 'ain',
        'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't",
        'as', 'at', 'be', 'because', 'been', 'before', 'being',
        'below', 'between', 'both', 'but', 'by', 'can', 'couldn',
        "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does',
        'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during',
        'each', 'few', 'for', 'from', 'further', 'had', 'hadn',
        "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't",
        'having', 'here', 'how', 'i', 'if', 'in', 'into', 'is', 'isn',
        "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm',
        'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn',
        "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor',
        'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or',
        'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own',
        're', 's', 'same', 'shan', "shan't", 'should', "should've",
        'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than',
        'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves',
        'then', 'there', 'these', 'they', 'this', 'those', 'through',
        'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn',
        "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where',
        'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won',
        "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll",
        "you're", "you've", 'your', 'yours', 'yourself', 'yourselves'}

In [4]:
m_path = filepath + 'models/'
v_path = filepath + 'matrices/'
if os.path.exists(m_path) == False:
    os.mkdir(m_path)

## Load Matrices

In [5]:
def load_matrices(v_path = v_path, size = sample_size):
    """
    Accepts path where term or tagged matrices are stored
    and creates a dictionary with all matrices for the 
    specified sample size.
    """
    matrix_dict = {}
    svd = TruncatedSVD(n_components = 100)
    for m in os.listdir(v_path):
        if '.wordvectors' not in m and size in m and 'train' in m:
            if m[-4:] == '.npz':
                name = m[:-4]
                matrix_dict[name] = svd.fit_transform(sparse.load_npz(v_path + m))
            elif m[-4:] == '.npy':
                name = m[:-4]
                matrix_dict[name] = np.load(v_path + m)
            else:
                name = m
                matrix_dict[name] = pd.read_csv(v_path + m).astype('int64')
    return matrix_dict

In [6]:
matrix_dict = load_matrices()
print(matrix_dict.keys())

dict_keys(['pos_m_X_train_one_million', 'cnt_m_X_train_one_million', 'tfidf_m_X_train_one_million', 'dep_m_X_train_one_million', 'w2v_m_X_train_one_million'])


## Load X_train & y_train

In [7]:
# load df's in dict iterable
def load_df_dict(size = sample_size, typ = None):
    """
    Accepts 'one', 'three', or 'five' as input sizes.
    If 'train', 'validate', or 'test' are inserted for type,
    only that kind of sample will be loaded.
    """
    df_dict = {}
    for sample in os.listdir(filepath + 'samples/'):
        if size in sample and (typ is None or typ in sample):
            name = sample.replace('.csv', '')
            temp_df = pd.read_csv(filepath + 'samples/' + sample)[['source', ' response_text', ' op_gender']]
            temp_df = temp_df.rename(columns = {' response_text': 'response_text', ' op_gender': 'op_gender'}).dropna()
            temp_df['y'] = np.where(temp_df.op_gender == 'W', 1, 0)
            df_dict['X_' + name] = temp_df.response_text
            df_dict['y_' + name] = temp_df.y
    return df_dict

In [8]:
df_dict = load_df_dict(typ = 'train')
X_train = df_dict[[key for key in df_dict.keys() if 'X_train' in key][0]]
y_train = df_dict[[key for key in df_dict.keys() if 'y_train' in key][0]]

## Create Evaluation Metrics

In [9]:
# create custom scoring function
def evaluator():
    accuracy = make_scorer(accuracy_score)
    precision = make_scorer(precision_score)
    recall = make_scorer(recall_score)
    adjusted_rand = make_scorer(adjusted_rand_score)
    adjusted_mutual_info = make_scorer(adjusted_mutual_info_score)
    silhouette = make_scorer(silhouette_score, greater_is_better = True)
    calinski_harabasz = make_scorer(calinski_harabasz_score, greater_is_better = True)
    return {'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'adjusted_rand': adjusted_rand,
            'adjusted_mutual_info': adjusted_mutual_info,
            'silhouette': silhouette,
            'calinski_harabasz': calinski_harabasz}
evals = evaluator()

scorers = ['accuracy', 'precision_weighted', 'recall_weighted',
           'adjusted_rand_score', 'adjusted_mutual_info_score']

## Create Documentation

In [10]:
# create path
d_path = filepath + 'documentation/'
if os.path.exists(d_path) == False:
    os.mkdir(d_path)
# if doc doesn't already exist, create it
doc_path = d_path + 'train_scores.csv'
if os.path.exists(doc_path) == False:
    train_doc_df = pd.DataFrame(columns = ['model_nm', 'matrix_nm', 'params', 'scores'])
    train_doc_df.to_csv(d_path + 'train_scores.csv', sep = '|', index = False)

In [11]:
# create function to check if matrix + model train combo already complete
def documenter(check_model, check_matrix, doc_path = doc_path):
    temp_df = pd.read_csv(doc_path, sep = '|')
    rows = temp_df[(temp_df.model_nm == check_model) & (temp_df.matrix_nm == check_matrix)].shape[0]
    if rows > 0:
        print(f'Training of {check_model} with {check_matrix} is already completed.')
        return False
    else: return True

In [12]:
# create function to write new results
def write_doc(model, matrix, params, scores, doc_path = doc_path):
    temp_df = pd.read_csv(doc_path, sep = '|')
    cols = list(temp_df.columns)
    append_df = pd.DataFrame([model, matrix, params, scores], index = cols).T
    temp_df = pd.concat([temp_df, append_df], ignore_index = True)
    temp_df.to_csv(doc_path, sep = '|', index = False)

In [13]:
# create looping function to fit and document
def looper(estimator, params, model_nm, scorers = scorers,
           y_train = y_train, matrix_dict = matrix_dict):
    for key, value in matrix_dict.items():
        if documenter(model_nm, key):
            print(f'*** Training {model_nm} with {key}.')
            gs = GridSearchCV(estimator, params, scoring = scorers, cv = 3,
                              n_jobs = -1, refit = False, verbose = 3)
            gs.fit(X = value, y = y_train)
            # write results to file
            result_dict = {key: value for key, value in gs.cv_results_.items() if 'mean_test_' in key}
            for i, param in enumerate(gs.cv_results_['params']):
                param_json = json.dumps(param, default = float)
                scores = json.dumps({key: value[i] for key, value in result_dict.items()})
                write_doc(model_nm, key, param, scores)

## Train KMeans

In [14]:
# create instance
km = KMeans(n_clusters = 2,
            random_state = random_state)
# create params to iterate over
params = {'max_iter': np.arange(2, 5) * 100,
          'tol'     : np.arange(9, 12) * 1e-5}
# remark model for documentation
model_nm = 'KMeans'
# run looper with above parameters
looper(estimator = km, params = params, model_nm = model_nm)

*** Training KMeans with pos_m_X_train_one_million.
Fitting 3 folds for each of 9 candidates, totalling 27 fits


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

*** Training KMeans with cnt_m_X_train_one_million.
Fitting 3 folds for each of 9 candidates, totalling 27 fits


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

*** Training KMeans with tfidf_m_X_train_one_million.
Fitting 3 folds for each of 9 candidates, totalling 27 fits


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

*** Training KMeans with dep_m_X_train_one_million.
Fitting 3 folds for each of 9 candidates, totalling 27 fits


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

*** Training KMeans with w2v_m_X_train_one_million.
Fitting 3 folds for each of 9 candidates, totalling 27 fits


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

## Train Hierarchical

In [15]:
# create instance
ac = AgglomerativeClustering(n_clusters = 2)
# create params to iterate over
params = [{'affinity' : ['euclidean'],
           'linkage'  : ['ward']},
          {'affinity' : ['euclidean', 'l1', 'l2', 'manhattan', 'cosine'],
           'linkage'  : ['complete', 'average', 'single']}]
# remark model for documentation
model_nm = 'Agglomerative'
looper(estimator = ac, params = params, model_nm = model_nm)

*** Training Agglomerative with pos_m_X_train_one_million.
Fitting 3 folds for each of 16 candidates, totalling 48 fits


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/cluster/_agglomerative.py", line 897, in fit
    out = memory.cache(tree_builder)(X, connectivity=connectivity,
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/joblib/memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/sw/pkgs/arc/

[CV 1/3] END max_iter=200, tol=0.00011; accuracy: (test=0.500) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=-0.001) precision_weighted: (test=0.498) recall_weighted: (test=0.500) total time=   4.1s
[CV 3/3] END max_iter=300, tol=0.00011; accuracy: (test=0.502) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.005) precision_weighted: (test=0.522) recall_weighted: (test=0.502) total time=  14.2s
[CV 1/3] END max_iter=200, tol=0.00011; accuracy: (test=0.500) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=-0.001) precision_weighted: (test=0.503) recall_weighted: (test=0.500) total time=   6.8s
[CV 2/3] END max_iter=300, tol=9e-05; accuracy: (test=0.504) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.015) precision_weighted: (test=0.521) recall_weighted: (test=0.504) total time=  18.1s
[CV 2/3] END affinity=l1, linkage=average; accuracy: (test=nan) adjusted_mutual_info_score: (test=nan) adjusted_rand_sco

/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 56, in _cached_call
    return cache[method]
KeyError: 'predict'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/_s

[CV 3/3] END max_iter=300, tol=0.0001; accuracy: (test=0.501) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.000) precision_weighted: (test=0.503) recall_weighted: (test=0.501) total time=   4.5s
[CV 2/3] END max_iter=200, tol=9e-05; accuracy: (test=0.496) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.011) precision_weighted: (test=0.478) recall_weighted: (test=0.496) total time=  10.6s
[CV 2/3] END max_iter=400, tol=0.00011; accuracy: (test=0.502) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.001) precision_weighted: (test=0.502) recall_weighted: (test=0.502) total time=   8.1s
[CV 3/3] END affinity=euclidean, linkage=ward; accuracy: (test=nan) adjusted_mutual_info_score: (test=nan) adjusted_rand_score: (test=nan) precision_weighted: (test=nan) recall_weighted: (test=nan) total time=   0.2s
[CV 1/3] END affinity=manhattan, linkage=single; accuracy: (test=nan) adjusted_mutual_info_score: (test=nan) adjusted_rand_sco

/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 56, in _cached_call
    return cache[method]
KeyError: 'predict'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/_s

*** Training Agglomerative with cnt_m_X_train_one_million.
Fitting 3 folds for each of 16 candidates, totalling 48 fits


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/cluster/_agglomerative.py", line 897, in fit
    out = memory.cache(tree_builder)(X, connectivity=connectivity,
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/joblib/memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/sw/pkgs/arc/

[CV 2/3] END max_iter=200, tol=9e-05; accuracy: (test=0.504) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.011) precision_weighted: (test=0.522) recall_weighted: (test=0.504) total time=  11.8s
[CV 2/3] END max_iter=400, tol=0.00011; accuracy: (test=0.496) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.011) precision_weighted: (test=0.478) recall_weighted: (test=0.496) total time=  10.0s
[CV 1/3] END max_iter=400, tol=0.00011; accuracy: (test=0.500) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=-0.001) precision_weighted: (test=0.503) recall_weighted: (test=0.500) total time=   6.7s
[CV 1/3] END max_iter=300, tol=9e-05; accuracy: (test=0.498) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.003) precision_weighted: (test=0.478) recall_weighted: (test=0.498) total time=  19.7s
[CV 2/3] END affinity=l2, linkage=single; accuracy: (test=nan) adjusted_mutual_info_score: (test=nan) adjusted_rand_score: 

KeyboardInterrupt: 

## Train Gaussian Mixture

In [16]:
# create instance
gm = GaussianMixture(n_components = 2,
                     random_state = random_state,
                     warm_start = True)
# create params to iterate over
params = {'covariance_type' : ['full', 'tied', 'diag', 'spherical'],
          'tol'             : np.arange(9, 12) * 1e-5,
          'max_iter'        : np.arange(50, 250, 50),
          'reg_covar'       : np.arange(9, 12) * 1e-7}
# remark model for documentation
model_nm = 'GMM'
looper(estimator = gm, params = params, model_nm = model_nm)

*** Training GMM with pos_m_X_train_one_million.
Fitting 3 folds for each of 144 candidates, totalling 432 fits


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

*** Training GMM with cnt_m_X_train_one_million.
Fitting 3 folds for each of 144 candidates, totalling 432 fits


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

*** Training GMM with tfidf_m_X_train_one_million.
Fitting 3 folds for each of 144 candidates, totalling 432 fits


...

/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))


[CV 2/3] END covariance_type=tied, max_iter=150, reg_covar=9e-07, tol=0.00011; accuracy: (test=0.499) adjusted_mutual_info_score: (test=-0.000) adjusted_rand_score: (test=-0.000) precision_weighted: (test=0.510) recall_weighted: (test=0.499) total time=  11.3s
[CV 3/3] END covariance_type=tied, max_iter=150, reg_covar=1.1e-06, tol=0.00011; accuracy: (test=0.500) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.000) precision_weighted: (test=0.513) recall_weighted: (test=0.500) total time=  11.2s
[CV 2/3] END covariance_type=tied, max_iter=200, reg_covar=1.1e-06, tol=9e-05; accuracy: (test=0.499) adjusted_mutual_info_score: (test=-0.000) adjusted_rand_score: (test=-0.000) precision_weighted: (test=0.510) recall_weighted: (test=0.499) total time=  11.4s
[CV 2/3] END covariance_type=diag, max_iter=50, reg_covar=1e-06, tol=0.00011; accuracy: (test=0.511) adjusted_mutual_info_score: (test=0.001) adjusted_rand_score: (test=0.133) precision_weighted: (test=0.515) recall_w

/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

[CV 2/3] END covariance_type=tied, max_iter=200, reg_covar=9e-07, tol=0.0001; accuracy: (test=0.499) adjusted_mutual_info_score: (test=-0.000) adjusted_rand_score: (test=-0.000) precision_weighted: (test=0.510) recall_weighted: (test=0.499) total time=  11.3s
[CV 3/3] END covariance_type=tied, max_iter=200, reg_covar=1.1e-06, tol=0.00011; accuracy: (test=0.500) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.000) precision_weighted: (test=0.513) recall_weighted: (test=0.500) total time=  11.4s
[CV 2/3] END covariance_type=diag, max_iter=50, reg_covar=1.1e-06, tol=0.0001; accuracy: (test=0.512) adjusted_mutual_info_score: (test=0.001) adjusted_rand_score: (test=0.134) precision_weighted: (test=0.515) recall_weighted: (test=0.512) total time=  28.9s
[CV 3/3] END covariance_type=diag, max_iter=100, reg_covar=1e-06, tol=0.00011; accuracy: (test=0.511) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.121) precision_weighted: (test=0.513) recall_wei

/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

## Train Bayesian Gaussian Mixture

In [17]:
# create instance
bgm = BayesianGaussianMixture(n_components = 2,
                              random_state = random_state,
                              warm_start = True)
# create params to iterate over
params = {'covariance_type' : ['full', 'tied', 'diag', 'spherical'],
          'tol'             : np.arange(9, 12) * 1e-5,
          'max_iter'        : np.arange(50, 250, 50),
          'reg_covar'       : np.arange(9, 12) * 1e-7}
# remark model for documentation
model_nm = 'BGMM'
looper(estimator = bgm, params = params, model_nm = model_nm)

*** Training BGMM with pos_m_X_train_one_million.
Fitting 3 folds for each of 144 candidates, totalling 432 fits


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))


[CV 3/3] END covariance_type=diag, max_iter=200, reg_covar=9e-07, tol=0.0001; accuracy: (test=0.511) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.121) precision_weighted: (test=0.513) recall_weighted: (test=0.511) total time=  27.7s
[CV 3/3] END covariance_type=spherical, max_iter=50, reg_covar=1e-06, tol=0.00011; accuracy: (test=0.500) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.000) precision_weighted: (test=0.516) recall_weighted: (test=0.500) total time=   5.7s
[CV 3/3] END covariance_type=spherical, max_iter=100, reg_covar=9e-07, tol=0.0001; accuracy: (test=0.500) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.000) precision_weighted: (test=0.516) recall_weighted: (test=0.500) total time=   5.8s
[CV 1/3] END covariance_type=spherical, max_iter=100, reg_covar=1.1e-06, tol=0.00011; accuracy: (test=0.499) adjusted_mutual_info_score: (test=-0.000) adjusted_rand_score: (test=-0.000) precision_weighted: (test=0.50

/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

[CV 1/3] END covariance_type=tied, max_iter=200, reg_covar=1e-06, tol=9e-05; accuracy: (test=0.499) adjusted_mutual_info_score: (test=-0.000) adjusted_rand_score: (test=-0.000) precision_weighted: (test=0.502) recall_weighted: (test=0.499) total time=  11.0s
[CV 2/3] END covariance_type=diag, max_iter=50, reg_covar=9e-07, tol=0.0001; accuracy: (test=0.512) adjusted_mutual_info_score: (test=0.001) adjusted_rand_score: (test=0.137) precision_weighted: (test=0.515) recall_weighted: (test=0.512) total time=  30.8s
[CV 1/3] END covariance_type=diag, max_iter=100, reg_covar=9e-07, tol=0.00011; accuracy: (test=0.488) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.141) precision_weighted: (test=0.486) recall_weighted: (test=0.488) total time=  27.4s
[CV 3/3] END covariance_type=diag, max_iter=150, reg_covar=9e-07, tol=0.00011; accuracy: (test=0.511) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.121) precision_weighted: (test=0.513) recall_weighted

...

/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

[CV 3/3] END covariance_type=full, max_iter=50, reg_covar=1e-06, tol=0.00011; accuracy: (test=0.512) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.151) precision_weighted: (test=0.513) recall_weighted: (test=0.512) total time= 3.8min
[CV 3/3] END covariance_type=full, max_iter=100, reg_covar=1.1e-06, tol=9e-05; accuracy: (test=0.513) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.162) precision_weighted: (test=0.513) recall_weighted: (test=0.513) total time= 6.7min
[CV 2/3] END covariance_type=full, max_iter=200, reg_covar=9e-07, tol=9e-05; accuracy: (test=0.509) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.081) precision_weighted: (test=0.509) recall_weighted: (test=0.509) total time= 7.9min
[CV 3/3] END covariance_type=tied, max_iter=200, reg_covar=9e-07, tol=0.00011; accuracy: (test=0.500) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.000) precision_weighted: (test=0.514) recall_weighted:

/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r

[CV 2/3] END covariance_type=full, max_iter=50, reg_covar=1.1e-06, tol=0.0001; accuracy: (test=0.509) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.079) precision_weighted: (test=0.509) recall_weighted: (test=0.509) total time= 3.8min
[CV 1/3] END covariance_type=full, max_iter=100, reg_covar=1e-06, tol=0.00011; accuracy: (test=0.488) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.135) precision_weighted: (test=0.487) recall_weighted: (test=0.488) total time= 4.3min
[CV 3/3] END covariance_type=full, max_iter=150, reg_covar=1.1e-06, tol=9e-05; accuracy: (test=0.513) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.162) precision_weighted: (test=0.513) recall_weighted: (test=0.513) total time= 6.6min
[CV 3/3] END covariance_type=full, max_iter=200, reg_covar=1.1e-06, tol=0.00011; accuracy: (test=0.513) adjusted_mutual_info_score: (test=0.000) adjusted_rand_score: (test=0.162) precision_weighted: (test=0.513) recall_weig

/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:390: RuntimeWarning: overflow encountered in long_scalars
  (tp + fp) * (fp + tn))
/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:389: RuntimeWarning: overflow encountered in long_scalars
  r